In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import random
import json
import jsonlines
import re
import string
import shutil
import os

FILTERING VIDEOS

In [2]:
keywords = {
    'climate change', 'global warming', 'greenhouse gases', 'rising sea levels',
    'extreme weather', 'temperature increase', 'climate action', 
    'carbon emissions', 'renewable energy', 'sustainable living',
    'climate science', 'climate crisis', 'climate adaptation',
    'natural disaster', 'extreme heat', 'melting ice cap', 'sea level rise',
    'biodiversity loss', 'deforestation', 'ocean acidification', 'climate policy', 
    'environmental policy',
    'disaster recovery', 'climate refugees',
    'carbon footprint', 'sustainable development', 'green technology',
    'renewable resource', 'eco-friendly',
    'ecosystem disruption', 'impact on climate',
    'paris agreement', 'climate awareness',
    'environmental justice', 'clean energy', 'zero carbon',
    'green infrastructure', 'ozone layer',
    'pollution', 'water scarcity', 'climate education'
}

In [3]:
#check if text or tags contain climate keywords:
def idiot_filter(video):
    return video['upload_date'] > '2015'


def about_climate(tags):
    tags_set = set(tags)
    return any(tag.strip().lower() in keywords for tag in tags_set)

def climate_text(text):
    lowercase_text = text.lower()
    return any(keyword in lowercase_text for keyword in keywords)

def climate_related(video):
    return idiot_filter(video) and (climate_text(video['title']) or about_climate(video['tags']) or climate_text(video['description']))

In [4]:
from pyspark import SparkContext
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/13 19:23:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
file_path = '/Volumes/Maxtor/yt_metadata_en.jsonl'

raw_data = sc.textFile(file_path)
dataset = raw_data.map(lambda x: json.loads(x))

In [6]:
recent_videos = dataset.filter(idiot_filter)
print("Filter recent video")
climate_videos = recent_videos.filter(climate_related)

Filter recent video


In [7]:
climate_videos.take(1)

#count = climate_videos.count()
#print("Number of rows after filtering:", count)

[{'categories': 'Science & Technology',
  'channel_id': 'UCzWQYUVCpZqtN93H8RR44Qw',
  'crawl_date': '2019-11-03 22:21:31.549534',
  'description': 'Climate change threatens to make dry regions even drier, so scientists at UC Berkeley created a device to make water out of thin air.\n» Subscribe to Seeker! http://bit.ly/subscribeseeker\n» Watch more Elements! http://bit.ly/ElementsPlaylist\n\nAt least one hundred million people live in desert regions around the world according to the UN, and they survive off of less than 25 cm of rainfall each year, and for many, even that minuscule water supply is under threat as the climate crisis is making dry areas even drier.\n\nSo scientists at UC Berkeley have been experimenting with materials that can pull drinking water out of thin air.\n\nThat’s right, right out of thin air.\n\nA chemist at the University of California, Berkeley reported that he and his colleagues have created a solar-powered device that could provide water to millions living i

In [8]:
climate_videos_path = "data/climate_videos.jsonl"

with jsonlines.open(climate_videos_path, "w") as jsonl_file:
    jsonl_file.write_all(climate_videos.collect())

#sc.stop()
#climate_videos.saveAsTextFile()
#climate_videos.saveAsTextFile('data/climate_videos.csv')"""

In [ ]:
climate_video_df = pd.read_csv(climate_videos_path)

SAMPLING

In [ ]:
#file_path = '/Volumes/Maxtor/youtube_comments.tsv'
#sample_size = 10000

In [ ]:
#df = pd.read_csv(file_path, sep='\t', header=0, nrows=sample_size)

In [ ]:
#df.to_csv('data/comments_sample.csv', index=False)